In [1]:
import pandas as pd 
import numpy as np

In [2]:
faults = pd.read_csv('../data/J1939Faults.csv')
onboard = pd.read_csv('../data/VehicleDiagnosticOnboardData.csv')

/tmp/ipykernel_6181/31407276.py:1: DtypeWarning: Columns (15) have mixed types. Specify dtype option on import or set low_memory=False.
  faults = pd.read_csv('../data/J1939Faults.csv')


In [3]:
onboard.dtypes

Id          int64
Name       object
Value      object
FaultId     int64
dtype: object

In [4]:
faults.dtypes

RecordID                   int64
ESS_Id                     int64
EventTimeStamp            object
eventDescription          object
actionDescription        float64
ecuSoftwareVersion        object
ecuSerialNumber           object
ecuModel                  object
ecuMake                   object
ecuSource                  int64
spn                        int64
fmi                        int64
active                      bool
activeTransitionCount      int64
faultValue               float64
EquipmentID               object
MCTNumber                  int64
Latitude                 float64
Longitude                float64
LocationTimeStamp         object
dtype: object

In [5]:
on_faults = pd.merge(faults, onboard, left_on = 'RecordID', right_on = 'FaultId').drop(columns = ['Id', 'FaultId'])

In [6]:


# Get rid of the rows where the EquipmentID is more than 5 characters
on_faults_filtered = on_faults.loc[on_faults['EquipmentID'].astype(str).str.len() <= 5]

# Filter out faults where the light is going off
on_faults_filtered = on_faults_filtered.loc[on_faults['active'] == True].reset_index(drop = True)



: 

: 

In [ ]:
on_faults_filtered['EquipmentID'].nunique()

In [ ]:
on_faults_filtered.dtypes

In [ ]:
# Convert EventTimeStamp column to datetime object
on_faults_filtered['EventTimeStamp'] = pd.to_datetime(on_faults_filtered['EventTimeStamp'])


In [ ]:
# Set EquipmentID and EventTimeStamp columns as index
on_faults_filtered.set_index(['EquipmentID', 'EventTimeStamp', 'spn'], inplace=True)

In [ ]:

# Pivot the data into a time series format
full_derates_ts = on_faults_filtered.pivot_table(index=['EquipmentID', 'EventTimeStamp', 'spn'],
                                          values=['RecordID', 'ESS_Id', 'eventDescription', 
                                                 'actionDescription', 'ecuSoftwareVersion', 'ecuSerialNumber', 'ecuModel', 
                                                 'ecuMake', 'ecuSource', 'fmi', 'active', 'activeTransitionCount', 
                                                 'faultValue', 'MCTNumber', 'Latitude', 'Longitude', 'LocationTimeStamp'])

# Reset the index
full_derates_ts.reset_index(inplace=True)

# Sort the data by EquipmentID and EventTimeStamp
full_derates_ts.sort_values(by=['EquipmentID', 'EventTimeStamp'], inplace=True)

# Group the data by EquipmentID
faults_grouped = full_derates_ts.groupby('EquipmentID')

# Create a dictionary of time series, with EquipmentID as the key and the time series as the value
faults_ts_dict = {group: data.drop('EquipmentID', axis=1).set_index('EventTimeStamp').sort_index() 
               for group, data in faults_grouped}


# Fill missing values with 0
for ts in faults_ts_dict.values():
    ts.fillna(0, inplace=True)



Now we have a dictionary where the keys are the specific Equipment Id's and the info inside of them pertians to the types of service codes that came up with a timestamp value. That way we can see if there is info preceding a 5246 "breakdown" code that could predict it coming ahead of time. 

Lets look at one of the trucks with the most data inculuded in its dictionary entry

In [ ]:
max_key = max(faults_ts_dict, key=lambda x: faults_ts_dict[x].shape[0])


In [ ]:
max_ts = faults_ts_dict.get(max_key)

In [ ]:
max_ts